In [ ]:
import cv2
import os
import numpy as np
import math
from keras.preprocessing.image import ImageDataGenerator

classes = ['Card','Goal', 'None', 'Substitution' ]

In [ ]:
# Create an ImageDataGenerator object
datagen = ImageDataGenerator(
    validation_split = 0.2,
    rotation_range=40,  # Degrees of random rotation
    width_shift_range = 0.2,  # % shift in horizontal direction
    height_shift_range = 0.2,  # % shift in vertical direction
    shear_range = 0.2,  # Shear angle in counter-clockwise direction in degrees
    rescale = 1./255, # Standardization
    horizontal_flip=True,  # Horizontal flipping
    fill_mode='nearest'  # Fill missing pixels using the 'nearest' pixel
)

In [ ]:
# Define the input directory and batch size
input_dir_train = '/content/drive/MyDrive/Major Project/Train'

# Generate augmented image data
image_generator_train = datagen.flow_from_directory(
    input_dir_train,
    batch_size=32,
    subset = 'training',
    target_size=(224, 224),  # Desired image size
    class_mode='categorical'  # Set the class mode based on your problem
    )


Found 2626 images belonging to 4 classes.


In [ ]:
image_generator_validation = datagen.flow_from_directory(
    input_dir_train,
    batch_size=8,
    subset = 'validation',
    target_size=(224, 224),  # Desired image size
    class_mode='categorical'
    )

Found 655 images belonging to 4 classes.


In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.losses import categorical_crossentropy

def VGG19(input_shape, class_num):

    model = Sequential()

    #Convolution Layers
    model.add(Conv2D(64, kernel_size=(3,3), padding= 'same',
                        activation= 'relu', input_shape= input_shape))
    model.add(Conv2D(64, kernel_size=(3,3), padding= 'same',
                    activation= 'relu'))

    #Pooling Layer
    model.add(MaxPooling2D(pool_size=(2,2), strides= (2,2)))


    #Convolution Layers
    model.add(Conv2D(128, kernel_size=(3,3), padding= 'same',
                    activation= 'relu'))
    model.add(Conv2D(128, kernel_size=(3,3), padding= 'same',
                    activation= 'relu'))

    #Pooling Layer
    model.add(MaxPooling2D(pool_size=(2,2), strides= (2,2)))

    #Convolution Layers
    model.add(Conv2D(256, kernel_size=(3,3), padding= 'same',
                    activation= 'relu'))
    model.add(Conv2D(256, kernel_size=(3,3), padding= 'same',
                    activation= 'relu'))
    model.add(Conv2D(256, kernel_size=(3,3), padding= 'same',
                    activation= 'relu'))
    model.add(Conv2D(256, kernel_size=(3,3), padding= 'same',
                    activation= 'relu'))

    #Pooling Layer
    model.add(MaxPooling2D(pool_size=(2,2), strides= (2,2)))

    #Convolution Layers
    model.add(Conv2D(512, kernel_size=(3,3), padding= 'same',
                    activation= 'relu'))
    model.add(Conv2D(512, kernel_size=(3,3), padding= 'same',
                        activation= 'relu'))
    model.add(Conv2D(512, kernel_size=(3,3), padding= 'same',
                    activation= 'relu'))
    model.add(Conv2D(512, kernel_size=(3,3), padding= 'same',
                    activation= 'relu'))

    #Pooling Layer
    model.add(MaxPooling2D(pool_size=(2,2), strides= (2,2)))

    #Convolution Layers
    model.add(Conv2D(512, kernel_size=(3,3), padding= 'same',
                    activation= 'relu'))
    model.add(Conv2D(512, kernel_size=(3,3), padding= 'same',
                    activation= 'relu'))
    model.add(Conv2D(512, kernel_size=(3,3), padding= 'same',
                    activation= 'relu'))
    model.add(Conv2D(512, kernel_size=(3,3), padding= 'same',
                    activation= 'relu'))

    #Pooling Layer
    model.add(MaxPooling2D(pool_size=(2,2), strides= (2,2)))

    #Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(4096, activation= 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation= 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(class_num, activation= 'softmax'))

    return model

model = VGG19((224,224,3), 4)


from keras import optimizers, losses, metrics

model.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                   

In [ ]:
trained_model = model.fit(image_generator_train, steps_per_epoch = 60, epochs=50, validation_data = image_generator_validation, validation_steps = 60)

Epoch 1/50
60/60 [==============================] - 1028s 17s/step - loss: 1.2951 - accuracy: 0.3359 - val_loss: 1.0752 - val_accuracy: 0.4958
Epoch 2/50
60/60 [==============================] - 292s 5s/step - loss: 0.9414 - accuracy: 0.6000 - val_loss: 0.9147 - val_accuracy: 0.6083
Epoch 3/50
60/60 [==============================] - 106s 2s/step - loss: 0.9662 - accuracy: 0.5947 - val_loss: 1.1361 - val_accuracy: 0.5417
Epoch 4/50
60/60 [==============================] - 61s 1s/step - loss: 0.8927 - accuracy: 0.5989 - val_loss: 0.8236 - val_accuracy: 0.6333
Epoch 5/50
60/60 [==============================] - 51s 844ms/step - loss: 0.8013 - accuracy: 0.6630 - val_loss: 0.8998 - val_accuracy: 0.6667
Epoch 6/50
60/60 [==============================] - 51s 852ms/step - loss: 0.7328 - accuracy: 0.6948 - val_loss: 0.8617 - val_accuracy: 0.6896
Epoch 7/50
60/60 [==============================] - 49s 825ms/step - loss: 0.6964 - accuracy: 0.7132 - val_loss: 0.7924 - val_accuracy: 0.6833
Epoch 

In [ ]:
from keras.models import load_model

model.save('/content/drive/MyDrive/Major Project/Model/model_vgg19_V2.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
